# 📊 Eval & Leaderboard Tracker – Module 15
Use this notebook to score completions, log results, and update your leaderboard.

## ✅ Step 1: Load Prompt + Completions

In [ ]:
prompt = "def add(x, y):"
completions = [
    "    return x + y",
    "    return x + y + 1",
    "    return x + y if x > 0 else 0"
]

## 🧠 Step 2: Score Each Completion

In [ ]:
from difflib import SequenceMatcher
import ast

def exact_match(c): return c.strip() == "return x + y"
def edit_dist(c): return SequenceMatcher(None, c.strip(), "return x + y").ratio()
def ast_valid(c): 
    try:
        ast.parse("def add(x, y):\n" + c)
        return True
    except:
        return False

scored = []
for i, c in enumerate(completions):
    scored.append({
        "index": i,
        "completion": c.strip(),
        "exact": exact_match(c),
        "edit_ratio": round(edit_dist(c), 3),
        "ast_ok": ast_valid(c)
    })

import pandas as pd
df = pd.DataFrame(scored)
df

## 📝 Step 3: Append to Leaderboard

In [ ]:
from datetime import datetime

entry = {
    "model": "test-checkpoint",
    "prompt": prompt,
    "exact_matches": int(df["exact"].sum()),
    "avg_edit_score": round(df["edit_ratio"].mean(), 3),
    "all_ast_valid": df["ast_ok"].all(),
    "datetime": datetime.now().isoformat()
}

import csv

lb_path = Path("data/leaderboard.csv")
lb_path.parent.mkdir(parents=True, exist_ok=True)

if not lb_path.exists():
    with lb_path.open("w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(entry.keys()))
        w.writeheader()
        w.writerow(entry)
else:
    with lb_path.open("a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(entry.keys()))
        w.writerow(entry)

print("✅ Leaderboard entry added.")